In [3]:
#!g1.1

DEBUG = False

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from numba import njit
import warnings

if DEBUG:
    how_much_read = 100
    step = 10
else:
    how_much_read = 2000
    step = 100
    

warnings.filterwarnings('ignore')

train = pd.read_csv("/home/jupyter/mnt/s3/hackathondata/train.csv").loc[:how_much_read] # убрать .loc[] если хотим считать все

train['sentiment'] = train['sentiment'].map({"+": 2, "?": 1, "−": 0})

from sklearn.model_selection import train_test_split
np.random.seed(2)
X_train, X_test, y_train, y_test = train_test_split(train["sentence"], train["sentiment"], test_size=0.3)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

nlp = spacy.load("ru_core_news_sm")

class KNN:    
    def fit(self, X, y):
        self.X = np.array([nlp(raw) for raw in X])
        self.y = y
        
    def set_k(self, k: int):
        self.k = k
    
    def predict(self, X):
        result = list()
        for raw in X:
            doc = nlp(raw)
            distances = np.array([doc.similarity(i) for i in self.X])
            max_dist_of_knn = np.partition(distances, self.k - 1)[self.k - 1]
            result.append(np.argmax(np.bincount(self.y[distances <= max_dist_of_knn])))
        return np.array(result)

def find_best_k_and_score(clf, X_test, y_test, k_iter, save):
    best_k = -1
    best_score = -1
    for k in k_iter:
        clf.set_k(k)
        pred = clf.predict(X_test)
        cur_score = accuracy_score(y_test, pred)
        save[k] = cur_score
        if cur_score > best_score:
            best_score = cur_score
            best_k = k
    return best_k
    
clf = KNN()
clf.fit(X_train, y_train)
save = dict()
k_iter = range(10, X_train.shape[0] // 2, step)
k = find_best_k_and_score(clf, X_test, y_test, k_iter, save)
print(f"k = {k}, accuracy = {save[k]}")

k = 610, accuracy = 0.49916805324459235


In [ ]:
#!g1.1
